In [1]:
from pyalex import config, Works, Authors, Sources, Institutions, Topics, Publishers, Funders
import helpers

In [2]:
# Email Configuration for the polite pool
config.email = "namemat@gmail.com"

In [3]:
# Configuration for retrying requests
config.max_retries = 3
config.retry_backoff_factor = 0.5  # delay between two retries
config.retry_https_codes = [429, 500, 503]

In [4]:
universities_ror_url = {
    # West Pomeranian University of Technology in Szczecin
    'PL_ZUT': 'https://ror.org/0596m7f19',
    # Burgas Free University
    'BG_BFU': 'https://ror.org/02ek1bx64',
    # University of Patras
    'GR_UOP': 'https://ror.org/017wvtq80',
    # University of Dubrovnik
    'HR_UNIDU': 'https://ror.org/05yptqp13',
    # University EMUNI
    'SL_EMUNI': 'https://ror.org/03761pf32',
    # University of Sassari
    'IT_UNISS': 'https://ror.org/01bnjbv91',
    # University of the Antilles
    'FR_UAG': 'https://ror.org/02ryfmr77',
    # University of the Azores
    'PT_UAC': 'https://ror.org/04276xd64',
    # University of the Balearic Islands
    'ES_UIB': 'https://ror.org/03e10x626',
    # University Le Havre Normandie
    'FR_ULHN': 'https://ror.org/05v509s40',
    # University of the Faroe Islands
    'FO_UF': 'https://ror.org/05mwmd090',
    # Stralsund University of Applied Sciences
    'DE_HOCHSTRALSUND': 'https://ror.org/04g99jx54',
    # Åland University of Applied Sciences
    'FI_AUAS': 'https://ror.org/05mknbx32',
}

universities_ror_id = {}
for key in universities_ror_url:
    universities_ror_id[key] = helpers.extract_id_from_url(universities_ror_url[key])

print(universities_ror_id)

{'PL_ZUT': '0596m7f19', 'BG_BFU': '02ek1bx64', 'GR_UOP': '017wvtq80', 'HR_UNIDU': '05yptqp13', 'SL_EMUNI': '03761pf32', 'IT_UNISS': '01bnjbv91', 'FR_UAG': '02ryfmr77', 'PT_UAC': '04276xd64', 'ES_UIB': '03e10x626', 'FR_ULHN': '05v509s40', 'FO_UF': '05mwmd090', 'DE_HOCHSTRALSUND': '04g99jx54', 'FI_AUAS': '05mknbx32'}


In [5]:
# NEEDS FIX as it's only getting the first page of results
def get_works_by_universities(ror_dict):
    result = {}
    for key in ror_dict:
        result[key] = Works().filter(authorships={"institutions": {
            "ror": ror_dict[key]}}).get()
    return result

res = get_works_by_universities(universities_ror_id)

In [6]:
# helpers.make_json_pretty(res)

In [7]:
len(res)
sum = 0
for key in res:
    sum += len(res[key])
    print(f'{key}: {len(res[key])}')
print(f'Total: {sum}')

PL_ZUT: 25
BG_BFU: 25
GR_UOP: 25
HR_UNIDU: 25
SL_EMUNI: 25
IT_UNISS: 25
FR_UAG: 25
PT_UAC: 25
ES_UIB: 25
FR_ULHN: 25
FO_UF: 25
DE_HOCHSTRALSUND: 25
FI_AUAS: 25
Total: 325


In [8]:
# def get_works_by_universities(ror_dict):
#     result = {}
#     pager = Works().filter(authorships={"institutions": {"ror": ror_dict['PL_ZUT']}}).paginate(per_page=100)
#     for page in pager:
#         if 'PL_ZUT' in result:
#             result['PL_ZUT'].extend(page)
#         else:
#             result['PL_ZUT'] = page
#     return result

# PL_ZUT = get_works_by_universities(universities_ror_id)
# print(len(PL_ZUT['PL_ZUT']))

In [11]:
all_works = []
error_count = 0
processed_count = 0
# try:
query = Works().filter(authorships={"institutions": {"ror": universities_ror_id['PL_ZUT']}})
for work in query.iterator():  # Use iterator() for memory efficiency
    processed_count += 1
    all_works.append(work)
    if processed_count % 500 == 0:  # Print progress periodically
        print(f"  Processed {processed_count} works...")

# except Exception as e:
#     print(f"\n--- An error occurred ---")
#     print(f"Error type: {type(e).__name__}")
#     print(f"Error details: {e}")
#     print(f"Processed {processed_count} works before the error.")
#     error_count += 1
#     # Depending on the error, you might want to implement retry logic here

print(f"\n--- Query Finished ---")
if error_count == 0:
    print(
        f"Successfully retrieved {len(all_works)} works for ROR PL_ZUT.")
else:
    print(f"Retrieved {len(all_works)} works before encountering an error.")

# --- Optional: Print some details of the first few works ---
if all_works:
    print("\n--- Sample Works (First 5) ---")
    for i, work in enumerate(all_works[:5]):
        work_id = work.get('id')
        title = work.get('title', 'N/A')
        doi = work.get('doi', 'N/A')
        pub_year = work.get('publication_year', 'N/A')
        print(f"{i+1}. ID: {work_id}")
        print(f"   Title: {title}")
        print(f"   DOI: {doi}")
        print(f"   Year: {pub_year}")
        print("-" * 10)
else:
    print("No works found or an error prevented retrieval.")

RecursionError: maximum recursion depth exceeded